# Final Results Compilation

Compile all results into a single summary and verify all numbers match between tables, figures, and text.


In [ ]:
import json
import os
import shutil

results_dir = '/kaggle/working/results'

# Load all results
files_to_load = [
    'jepa_results.json',
    'original_results.json',
    'baseline_results.json',
    'ablation_results.json',
    'sample_efficiency_results.json',
    'efficiency_results.json',
    'gate_analysis_results.json',
    'case_studies.json'
]

all_results = {}
for fname in files_to_load:
    fpath = os.path.join(results_dir, fname)
    if os.path.exists(fpath):
        with open(fpath, 'r', encoding='utf-8') as f:
            all_results[fname.replace('.json', '')] = json.load(f)
        print(f"[OK] Loaded {fname}")
    else:
        print(f"[MISSING] {fname}")

# ============================================================
# VERIFICATION CHECKLIST
# ============================================================
print("\n" + "=" * 60)
print("VERIFICATION CHECKLIST")
print("=" * 60)

checks = []

# Check 1: All methods have results
required_methods = ['mcda', 'mlp', 'neural_ranker', 'din',
                    'original', 'jepa']
for method in required_methods:
    has_result = any(method in str(v) for v in all_results.values())
    status = '[OK]' if has_result else '[FAIL]'
    checks.append(f"{status} Results for {method}")
    print(checks[-1])

# Check 2: All metrics computed
required_metrics = ['ndcg@1', 'ndcg@5', 'ndcg@10', 'map', 'mrr', 'hr@5']
for metric in required_metrics:
    has_metric = metric in str(all_results.get('jepa_results', {}))
    status = '[OK]' if has_metric else '[FAIL]'
    checks.append(f"{status} Metric: {metric}")
    print(checks[-1])

# Check 3: Multiple seeds
jepa = all_results.get('jepa_results', {})
has_runs = 'runs' in str(jepa)
status = '[OK]' if has_runs else '[WARN]'
checks.append(f"{status} Multiple seed runs recorded")
print(checks[-1])

# Check 4: Ablation studies complete
ablation = all_results.get('ablation_results', {})
required_ablations = ['no_gates', 'no_twostage', 'mse_loss',
                      'ranking_loss', 'symmetric_lr', 'no_vicreg',
                      'gate_dim_16', 'gate_dim_64']
for abl in required_ablations:
    has_abl = abl in str(ablation)
    status = '[OK]' if has_abl else '[FAIL]'
    checks.append(f"{status} Ablation: {abl}")
    print(checks[-1])

# Check 5: Statistical tests
gate_analysis = all_results.get('gate_analysis_results', {})
has_stats = 'kruskal' in str(gate_analysis) or 'p_value' in str(gate_analysis)
status = '[OK]' if has_stats else '[FAIL]'
checks.append(f"{status} Statistical significance tests")
print(checks[-1])

# Check 6: Stratified results
has_stratified = 'stratified' in str(all_results.get('jepa_results', {}))
status = '[OK]' if has_stratified else '[FAIL]'
checks.append(f"{status} Stratified results by context")
print(checks[-1])

# ============================================================
# SUMMARY TABLE FOR PAPER ABSTRACT
# ============================================================
print("\n" + "=" * 60)
print("NUMBERS FOR PAPER ABSTRACT")
print("=" * 60)

jepa_overall = all_results.get('jepa_results', {}).get('overall', {})
orig_overall = all_results.get('original_results', {}).get('overall', {})
mcda_overall = all_results.get('baseline_results', {}).get('mcda', {}).get('overall', {})

improvement_vs_original = {}
improvement_vs_mcda = {}

if jepa_overall and orig_overall:
    for metric in ['ndcg@5', 'map', 'mrr']:
        jepa_val = jepa_overall.get(metric, {}).get('mean', 0)
        orig_val = orig_overall.get(metric, {}).get('mean', 0)
        mcda_val = mcda_overall.get(metric, {}).get('mean', 0)

        improve_vs_orig = ((jepa_val - orig_val) / orig_val * 100) if orig_val > 0 else 0
        improve_vs_mcda = ((jepa_val - mcda_val) / mcda_val * 100) if mcda_val > 0 else 0

        improvement_vs_original[metric] = improve_vs_orig
        improvement_vs_mcda[metric] = improve_vs_mcda

        print(f"\n{metric.upper()}:")
        print(f"  JEPA:     {jepa_val:.4f}")
        print(f"  Original: {orig_val:.4f} (JEPA is {improve_vs_orig:+.1f}% better)")
        print(f"  MCDA:     {mcda_val:.4f} (JEPA is {improve_vs_mcda:+.1f}% better)")

# ============================================================
# COMPILE ALL FOR DOWNLOAD
# ============================================================

# Create a complete results package
package_dir = '/kaggle/working/paper_package'
os.makedirs(f'{package_dir}/tables', exist_ok=True)
os.makedirs(f'{package_dir}/figures', exist_ok=True)
os.makedirs(f'{package_dir}/data', exist_ok=True)

# Copy tables (if present)
tables_dir = f'{results_dir}/tables'
if os.path.isdir(tables_dir):
    for f in os.listdir(tables_dir):
        shutil.copy(f'{tables_dir}/{f}', f'{package_dir}/tables/')
else:
    print('[WARN] tables directory not found')

# Copy figures (if present)
figures_dir = f'{results_dir}/figures'
if os.path.isdir(figures_dir):
    for f in os.listdir(figures_dir):
        shutil.copy(f'{figures_dir}/{f}', f'{package_dir}/figures/')
else:
    print('[WARN] figures directory not found')

# Copy result JSONs
for f in files_to_load:
    src = os.path.join(results_dir, f)
    if os.path.exists(src):
        shutil.copy(src, f'{package_dir}/data/')

# Create summary
summary = {
    'verification': checks,
    'key_results': {
        'jepa_ndcg5': jepa_overall.get('ndcg@5', {}),
        'improvement_vs_original': improvement_vs_original,
        'improvement_vs_mcda': improvement_vs_mcda
    }
}

with open(f'{package_dir}/results_summary.json', 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2)

# Zip it
shutil.make_archive('/kaggle/working/paper_results_package', 'zip', package_dir)

print("\n[OK] Complete results package saved!")
print('Download: /kaggle/working/paper_results_package.zip')
print(f"\nAll {sum('[OK]' in c for c in checks)}/{len(checks)} checks passed")

